In [ ]:
import json
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import torch
import torchvision
from  torchvision import transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import random
from array2gif import write_gif
from PIL import Image
import IPython
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from IPython import display
from pathlib import Path
import time
import tqdm
from tqdm.notebook import trange, tqdm
import math
from torch import autograd
import matplotlib.animation as animation

## Get Images

In [ ]:
#10 is bubble gum
img_num = 10

In [ ]:
img_folder = "64"

In [ ]:
enteries = os.listdir(img_folder)

In [ ]:
img = mpimg.imread("64/"+enteries[img_num])

In [ ]:
plt.imshow(img)

In [ ]:
np.array(img).shape

In [ ]:
temp_img = Image.fromarray(np.uint8(img*255.))
plt.imshow(temp_img)

In [ ]:
temp_img = Image.fromarray(np.uint8(img*255))
plt.imshow(temp_img)

In [ ]:
def rgba_to_rgb(img):
    temp_img = Image.fromarray(np.uint8(img*255))
    background = Image.new("RGB",(64,64), (255,255,255))
    print(background.size)
    background.paste(temp_img,mask=temp_img.split()[3])
    background = np.array(background)
    print(type(background))
    background = (background - background.min())/(background.max()-background.min())
    return background

In [ ]:
img = rgba_to_rgb(img)
plt.imshow(img)
print(img.shape)
print(img[0][0])

## state to image

In [ ]:
def state_to_image(state,num_channels):
    state = state.permute(3,2,1,0)
    state = torch.reshape(state,(64,64,1,num_channels))
    state = torch.reshape(state,(64,64,num_channels))
    state = state.narrow(2,0,3)
    return state

## Build Neural Network

In [ ]:
class big_NN(nn.Module):
    def __init__(self, num_channels):
        super(big_NN, self).__init__()
        temp = num_channels*3
        self.fc1 = nn.Linear(temp, temp)
        #self.fc4 = nn.Linear(temp, temp)
        self.fc2 = nn.Linear(temp, num_channels)
        #self.fc3 = nn.Tanh()
        
    def forward(self,x):
        x1 = F.relu(self.fc1(x))
        #x4 = F.relu(self.fc4(x1))
        x2 = self.fc2(x1)*.05
        return x2

## Initial states

In [ ]:
def initial_state(num_channels):
    state = torch.zeros((num_channels,1,64,64))
    for channel in range(num_channels):
        for i in range(64):
            for j in range(64):
                if(i == 31 and j == 31):
                    state[0][0][31][31] = 1.
                    state[1][0][31][31] = 1.
                    state[2][0][31][31] = 1.
                    state[3][0][31][31] = 1.
    return state

In [ ]:
def initial_random_state():
    inp_channels = 16
    out_channels = 16
    random_seeds = 15
    state = torch.zeros((inp_channels,1,64,64))
    for i in range(random_seeds):
        rand_x = random.randint(0,63)
        rand_y = random.randint(0,63)
        for channel in range(inp_channels):
            state[channel][0][rand_x][rand_y] = .5
    return state

In [ ]:
def initial_perturbed_state(img):
    state_img = np.reshape(np.transpose(img),(3,64,64))
    state = np.ones((16,1,64,64))
    state[0][0] = state_img[0]
    state[1][0] = state_img[1]
    state[2][0] = state_img[2]
    state = torch.FloatTensor(state)
    return state

In [ ]:
plt.imshow(img)

In [ ]:
print(img.shape)

In [ ]:
state1 = np.reshape(np.transpose(img),(3,64,64)) 
print(state1.shape)

In [ ]:
state2 = np.reshape(np.transpose(state1),(64,64,3)) 
print(state2.shape)

In [ ]:
plt.imshow(state2)

## Perception

In [ ]:
def perception(state):
    sobel_x = torch.FloatTensor([[-1,0,1],[-2,0,2],[-1,0,1]])
    sobel_y = np.transpose(sobel_x)
    
    sobel_x = sobel_x.view(1,1,3,3)
    sobel_y = sobel_y.view(1,1,3,3)
    
    grad_x = F.conv2d(state, sobel_x,stride=1,padding=1)
    grad_y = F.conv2d(state, sobel_y,stride=1,padding=1)
    
    temp_state = torch.cat((state,grad_x,grad_y),0)
    
    return temp_state

## Stochastic Update

In [ ]:
def stochastic_update(state,state_update):
    state_update = state_update.permute(1,0)
    state_update = torch.reshape(state_update,(num_channels,1,64,64))     #9,2 --> 2,1,3,3
    return state_update
    update_mask = torch.FloatTensor(num_channels,1,64,64).uniform_()>.1
    state_update = torch.mul(state_update,update_mask)
    return state_update

## Alive Masking

In [ ]:
def alive_masking(state,state_update):
    
    blur_state = F.max_pool2d(state[3],kernel_size=3,stride=1,padding=1)
    
    alive = torch.where((blur_state>.1),state_update,torch.zeros((num_channels,1,64,64)))  #not sure about this

    state = state + alive
    state = torch.where((state>=1.),torch.ones((num_channels,1,64,64)),state)
    state = torch.where((state<=0.),torch.zeros((num_channels,1,64,64)),state)
    
    return state + alive 
    
    

## Train

In [ ]:
buffer = []
num_channels = 20
model = big_NN(num_channels)

In [ ]:
epochs = 700
mse = nn.MSELoss()
l1_crit = nn.L1Loss(size_average=False)
optimizer = optim.Adam(model.parameters(), lr=0.01)
images = []
loss_list = []
img = torch.FloatTensor(img)
for epoch in range(epochs):
    state = initial_state(num_channels) 
    
    steps = 160
    for step in range(steps):
        conv_state = perception(state)
        conv_state = torch.reshape(conv_state.permute(3,2,1,0),(1*64*64,num_channels*3))
        state_update = model(conv_state)
        state_update = stochastic_update(state,state_update)

        state = alive_masking(state,state_update)
    temp_state = state_to_image(state,num_channels)
    
    
    
    sum_avg = 0
    
    loss = mse(temp_state,img) #+ sum_avg
    loss_list.append(float(loss))
    optimizer.zero_grad()
    model.zero_grad()
    loss.backward(retain_graph=True)
    for i in range(1):
        optimizer.step()
    #print("Epoch: {}    loss: {}  ran_num: {} state_avg: {}".format(epoch,loss.detach(),ran_num,state_sum_avg.detach()))
    print("Epoch: {}    loss: {}   avg: {}".format(epoch,loss.detach(),sum_avg))


In [ ]:
plt.imshow(temp_state.detach().numpy())

In [ ]:
plt.imshow(img)

In [ ]:
plt.plot([i for i in range(len(loss_list))],loss_list)

In [ ]:
inp_channels = 64
out_channels = 64
images = []

state = initial_state(num_channels) 
steps = 160
for step in range(steps):

    conv_state = perception(state)
    conv_state = torch.reshape(conv_state.permute(3,2,1,0),(1*64*64,num_channels*3))
    state_update = model(conv_state)
    state_update = stochastic_update(state,state_update)

    state = alive_masking(state,state_update)
    temp_state = state_to_image(state,num_channels)
    temp_state = (temp_state - temp_state.min())/(temp_state.max() - temp_state.min())
    images.append(temp_state.detach().numpy())
loss = mse(temp_state,img)
print("Epoch: {}    loss: {}".format(epoch,loss.detach()))


In [ ]:
print(len(images))

In [ ]:
image_list = [Image.fromarray(np.uint8(image*255.)) for image in images]

In [ ]:
image_list = [image.resize((640,640)) for image in image_list]
print(image_list[0].size)

In [ ]:
image_list[0].save('growing_phase.gif',save_all=True,append_images=image_list[1:],optimize=False,duration=40,loops=10)

In [ ]:
gifPath = Path("growing_phase.gif")
with open(gifPath,'rb') as f:
    display.Image(data=f.read(),format='png')

In [ ]:
'''    #randomly change state initilaization
    ran_num = random.randint(0,2)
    if(ran_num == 0):
        state = initial_state()
    elif(ran_num == 1):
        state = initial_random_state()
    elif(ran_num == 2):
        state = initial_perturbed_state(img)
    else:
        state = initial_state() '''

In [ ]:
'''
#loss and backprop
param_sum_avg = 0
count = 0
for param in next(model.parameters()):
    count+=1
    param_sum_avg += param.mean()
param_sum_avg = param_sum_avg/count

state_sum_avg = 0
count = 0
for val in state:
    count+=1
    state_sum_avg += val.mean()
state_sum_avg = state_sum_avg/count
#print(state_sum_avg)
'''

In [ ]:
'''
for param in next(model.parameters()):
    #print(param)
    temp_sum = 0
    for i in param:
        temp_sum += abs(i)
        count+=1             
    sum_avg += temp_sum
    
sum_avg = sum_avg/count
print("Sum avg param: {}".format(sum_avg))
'''